# netCDF - Working with Network Common Data Form Datasets
### Omodayo Origunwa 
**Deliverable**
- [View Project Current State](https://github.com/dino68/netCDF-Data-Visualization)

**Goals**
* Using Jupyter notebook and Python, generate a tool to fulfill each of the following requests respectively: 
    * netCDF data acquisition
    * netCDF data analysis
    * netCDF data visualization
* Leverage **pyenv** and **virtualenv** or conda to support coding stability
* Adhere to python coding best-practices and demonstrate proficiency with modular, object oriented coding with a focus on reproducibility.

#### To Do:
1. netCDF data acquisition tooling
2. netCDF data analysis tooling

#### In Progress:
3. netCDF data visualization tooling

### netCDF Data Visualization Tools: 

In [1]:
# Import Libraries
import numpy as np 
import netCDF4 as nc
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap, addcyclic
from scipy.ndimage.filters import minimum_filter, maximum_filter

In [2]:
# Define modular solution
class DataPlotter:
    
    def __init__(self):
        self.name = "DataPlotter"
        self.datasets = []

    def __str__(self):
        return "netCDF4 Data Plotter"
    
    def load_data(self, *args):
        for arg in args:
            self.datasets.append(nc.Dataset(arg))
        self.view_datasets()
    
    def view_datasets(self):
        for ds in self.datasets:
            print(f"\n\n\nRaw Dataset: \n{ds}\n")
            keys = ds.variables.keys()
            print(f"Dataset Keys: \n{keys}\n")
            for key in keys:
                var_data = ds.variables[key]
                var_size = len(var_data)
                print(f"\nVariable Name: {key}")
                print(f"Variable Size: {var_size}")
                print(f"Variable Shape: {var_data.shape}")
                print(f"Variable Dimensions: {var_data.dimensions}\n")   
                print(f"Raw Variable Data: {var_data}\n")
                """
                for index in range(var_size):
                    var_inst = var_data[index]
                    if type(var_inst) is np.ma.core.MaskedArray:
                        print("Found a Masked Array!")
                        print(f"Var: {key}, index: {index}")
                """
     
                    
    def get_datasets(self):
        return self.datasets                
    
    def get_extrema(self, mat, mode='wrap',window=10):
        """
        find the indices of local extrema (min and max)
        in the input array.
        src: https://matplotlib.org/basemap/users/examples.html
        """
        mn = minimum_filter(mat, size=window, mode=mode)
        mx = maximum_filter(mat, size=window, mode=mode)
        # (mat == mx) true if pixel is equal to the local max
        # (mat == mn) true if pixel is equal to the local in
        # Return the indices of the maxima, minima
        return np.nonzero(mat == mn), np.nonzero(mat == mx)
    
    def gen_basemap(self, ds):
        # overlay political map boundaries
        lons = ds.variables['longitude'][:]
        lats = ds.variables['latitude'][:]
        low_lvls = ds.variables['lowlevel'][:]
        m = Basemap(projection='mill',
                     llcrnrlon=min(lons),   # lower longitude
                     llcrnrlat=min(lats),    # lower latitude
                     urcrnrlon=max(lons),   # uppper longitude
                     urcrnrlat=max(lats))   # uppper latitude
        m.drawcoastlines()
        #m.drawstates() # seems to require balanced input
        m.drawcountries()
        m.drawlsmask(land_color='Linen', ocean_color='#CCFFFF') 
        m.drawcounties()  
        return m, lons, lats, low_lvls
    
    def plot_data(self, ds):
        # X-axis represents longitude (in deg)
        # Y-axis represents latitude (in deg)
        # Generate Basemap & Formst graph data
        bMap, lons, lats, low_lvls = self.gen_basemap(ds)
        #low_lvl_xtrma = [(self.get_extrema(lvl_set)) for lvl_set in low_lvls]

        nlats = len(lats)
        nlons = len(lons)
        lon, lat = np.meshgrid(lons, lats)
        
        # set axis
        x,y = bMap(lon,lat)

        nlats = len(lats)
        nlons = len(lons)
        
        """
        
        Everything below this line is a Frankenstein from the sea level pressure
        example on the netCDF website.
        
        # the window parameter controls the number of highs and lows detected.
        # (higher value, fewer highs and lows)
        # create Basemap instance.
        
        prmsl, lons = addcyclic(lats, lons)
        local_min, local_max = self.get_extrema(prmsl, mode='wrap', window=50)
        # contour levels
        clevs = np.arange(900,1100.,5.)
        # find x,y of map projection grid.
        # create figure.
        fig=plt.figure(figsize=(8,4.5))
        ax = fig.add_axes([0.05,0.05,0.9,0.85])
        cs = bMap.contour(x,y,prmsl,clevs,colors='k',linewidths=1.)
        bMap.drawcoastlines(linewidth=1.25)
        bMap.fillcontinents(color='0.8')
        #m.drawparallels(np.arange(-80,81,20),labels=[1,1,0,0])
        #m.drawmeridians(np.arange(0,360,60),labels=[0,0,0,1])
        xlows = x[local_min]; xhighs = x[local_max]
        ylows = y[local_min]; yhighs = y[local_max]
        lowvals = prmsl[local_min]; highvals = prmsl[local_max]
        # plot lows as blue L's, with min pressure value underneath.
        xyplotted = []
        # don't plot if there is already a L or H within dmin meters.
        yoffset = 0.022*(bMap.ymax-bMap.ymin)
        dmin = yoffset
        for x,y,p in zip(xlows, ylows, lowvals):
            if x < bMap.xmax and x > bMap.xmin and y < bMap.ymax and y > bMap.ymin:
                dist = [np.sqrt((x-x0)**2+(y-y0)**2) for x0,y0 in xyplotted]
                if not dist or min(dist) > dmin:
                    plt.text(x,y,'L',fontsize=14,fontweight='bold',
                            ha='center',va='center',color='b')
                    plt.text(x,y-yoffset,repr(int(p)),fontsize=9,
                            ha='center',va='top',color='b',
                            bbox = dict(boxstyle="square",ec='None',fc=(1,1,1,0.5)))
                    xyplotted.append((x,y))
        # plot highs as red H's, with max pressure value underneath.
        xyplotted = []
        for x,y,p in zip(xhighs, yhighs, highvals):
            if x < bMap.xmax and x > bMap.xmin and y < bMap.ymax and y > bMap.ymin:
                dist = [np.sqrt((x-x0)**2+(y-y0)**2) for x0,y0 in xyplotted]
                if not dist or min(dist) > dmin:
                    plt.text(x,y,'H',fontsize=14,fontweight='bold',
                            ha='center',va='center',color='r')
                    plt.text(x,y-yoffset,repr(int(p)),fontsize=9,
                            ha='center',va='top',color='r',
                            bbox = dict(boxstyle="square",ec='None',fc=(1,1,1,0.5)))
                    xyplotted.append((x,y))
        plt.title('Mean Sea-Level Pressure (with Highs and Lows) %s' % date)
        plt.show()
        return 1
        """
    


In [3]:
# Import Files (assumed local)
conus = 'lowlevel_conus_2017-03-07T00_00_00Z_to_2017-03-07T00_05_00Z.nc'
conus_dbz = 'lowlevel_dbz_hourly_conus_2017-03-07T00_00_00Z_to_2017-03-07T01_00_00Z.nc'

In [4]:
# Plan, Do, Check, Adjust
dp = DataPlotter()

In [5]:
dp.load_data(conus, conus_dbz)
datasets = dp.get_datasets()




Raw Dataset: 
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): longitude(6201), latitude(3001)
    variables(dimensions): float64 longitude(longitude), float64 latitude(latitude), float64 lowlevel(longitude, latitude)
    groups: 

Dataset Keys: 
dict_keys(['longitude', 'latitude', 'lowlevel'])


Variable Name: longitude
Variable Size: 6201
Variable Shape: (6201,)
Variable Dimensions: ('longitude',)

Raw Variable Data: <class 'netCDF4._netCDF4.Variable'>
float64 longitude(longitude)
unlimited dimensions: 
current shape = (6201,)
filling on, default _FillValue of 9.969209968386869e+36 used


Variable Name: latitude
Variable Size: 3001
Variable Shape: (3001,)
Variable Dimensions: ('latitude',)

Raw Variable Data: <class 'netCDF4._netCDF4.Variable'>
float64 latitude(latitude)
unlimited dimensions: 
current shape = (3001,)
filling on, default _FillValue of 9.969209968386869e+36 used


Variable Name: lowlevel
Variable Size: 6201

In [6]:
# Plot netCDF data on selected Basemap(s)
#dp.plot_data(datasets[0])
#dp.plot_data(datasets[1])

## Step 5: Troubleshoot Mental Model

### Open Questions...
1. How does one go about evaluating the appropriate Basemap for a given netCDF Dataset? Currently struggling to identify a source of documentation listing the inputs and outputs for each of these basemap opts. 

2. How is pressure calcualted only from longitudinal and latitudinal data? Judging from the Dataset descriptions, the lowlevel variable is a masked array of tuples containing longitudinal and latitudinal data. (Although when I manually printed out each value they all seem to be '0.0'...) 
    * Also there are units listed with the second dataset's lowlevel data but not the first. Why is that?

3. The 'drawstates()' netCDF method seems to require a 'balanced' input, ie. len(x) = len(y) = len(z). 

4. Currently trying to understand the mask applied to each dataset, perhaps shifting focus from Data Visualization to building the Data Acquisition tools will prove to be more fruitful.

**Sources**:

_01.17.2020_
* The Austrailian Government coming in clutch: https://www.longpaddock.qld.gov.au/silo/gridded-data/python/
* netCDF Software Solutions (For sanity checks): https://www.unidata.ucar.edu/software/netcdf/software.html
* Good Folks over at ResearchGate: https://www.researchgate.net/post/How-can-we-mask-the-NetCDF-data-file-using-the-shapefile-of-our-area-of-interest-using-matlab

_01.15.2020_
* netCDF4 Data Model: https://www.unidata.ucar.edu/software/netcdf/docs/netcdf_data_set_components.html
* useful blog: https://joehamman.com/2013/10/12/plotting-netCDF-data-with-Python/
* matplotlib docs: https://matplotlib.org/basemap/users/examples.html
* netCDF examples: https://ccsr.aori.u-tokyo.ac.jp/~masakazu/memo/ncutil/nc.py
* netCDF Lecture: https://www.youtube.com/watch?v=d6diNBEDPfM
* BEST LEAD: https://stackoverflow.com/questions/39543869/replacing-values-in-array-from-netcdf

#### netCDF Basemap opts -> a dict (k,v) map for this + wrapper to optimize map selection would be fantastic

 **cyl:**              Cylindrical Equidistant               
 **merc:**             Mercator                                
 **tmerc:**            Transverse Mercator                     
 **omerc:**           Oblique Mercator                        
 **mill:**             Miller Cylindrical                      
 **gall:**             Gall Stereographic Cylindrical          
 **cea:**              Cylindrical Equal Area                  
 **lcc:**              Lambert Conformal                       
 **laea:**             Lambert Azimuthal Equal Area            
 **nplaea:**           North-Polar Lambert Azimuthal           
 **splaea:**           South-Polar Lambert Azimuthal           
 **eqdc:**             Equidistant Conic                       
 **aeqd:**             Azimuthal Equidistant                   
 **npaeqd:**           North-Polar Azimuthal Equidistant       
 **spaeqd:**           South-Polar Azimuthal Equidistant       
 **aea:**              Albers Equal Area                       
 **stere:**            Stereographic                           
 **npstere:**          North-Polar Stereographic               
 **spstere:**          South-Polar Stereographic               
 **cass:**             Cassini-Soldner                         
 **poly:**             Polyconic                               
 **ortho:**            Orthographic                            
 **geos:**             Geostationary                           
 **nsper:**            Near-Sided Perspective                  
 **sinu:**             Sinusoidal                              
 **moll:**             Mollweide                               
 **hammer:**           Hammer                                  
 **robin:**            Robinson                                
 **kav7:**             Kavrayskiy VII                          
 **eck4:**             Eckert IV                               
 **vandg:**            van der Grinten                         
 **mbtfpq:**           McBryde-Thomas Flat-Polar Quartic       
 **gnom:**             Gnomonic                                
 **rotpole:**          Rotated Pole